In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import psutil
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
import dask.array as da
import zarr
import pylops
import pylops_distributed

from dask_jobqueue import PBSCluster
from dask.distributed import Client

In [2]:
cluster = PBSCluster(cores=12,
                     memory='100GB',
                     shebang='#!/bin/bash',
                     resource_spec='nodes=1:baloo', #'nodes=1:ppn=16', #'select=1:baloo',
                     queue='normal',
                     walltime='12:00:00',
                     project='account')

In [3]:
print(cluster.job_script())

#!/bin/bash

#PBS -N dask-worker
#PBS -q normal
#PBS -A account
#PBS -l nodes=1:baloo
#PBS -l walltime=12:00:00
JOB_ID=${PBS_JOBID%%.*}

/private/mrava/miniconda3/envs/pylops-distributed/bin/python -m distributed.cli.dask_worker tcp://10.14.75.123:37716 --nthreads 12 --memory-limit 100.00GB --name name --nanny --death-timeout 60



In [4]:
cluster.scale(jobs=3)

In [5]:
client = Client(cluster)

In [6]:
client

Client Scheduler: tcp://10.14.75.123:37716 Dashboard: http://10.14.75.123:8787/status,Cluster Workers: 3 Cores: 36 Memory: 300.00 GB


In [7]:
A = da.zeros((1000, 2000), chunks=(100, 100))
x = da.zeros(2000, chunks=100)

y= da.dot(A, x)

In [8]:
y.compute()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [9]:
client.close()

In [10]:
cluster.close()